# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902024


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.42s/it]

Rendering models:  10%|█         | 3/30 [00:03<00:28,  1.06s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:40,  1.55s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:29,  1.17s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  4.41it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.84it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  3.54it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  2.77it/s]

CTidwell3                             0.000347
firejuggler                           0.000400
BeNjEsSePh                            0.000389
not-logged-in-cae133427d99aac1e554    0.000498
kayleebug2017                         0.001392
JessieDawn                            0.003266
not-logged-in-9f66093d0c2b8fe22169    0.000790
nickoftona                            0.000703
equidad1                              0.088890
not-logged-in-8c0c44faf754d344b18a    0.016742
Cadetrnd                              0.001071
Marra                                 0.001740
not-logged-in-5f30b2bb9b7f9d49da41    0.256836
not-logged-in-5f30b2bb9b7f9d49da41    0.007328
peakscience7                          0.025327
acapirala                             0.002027
not-logged-in-04ed9ae880f18762cd9e    2.194296
Lavadude                              0.058317
ZoshiePoshie                          0.000477
Peyton.Harvey                         0.897724
not-logged-in-fc0fffd0b7fe097394c8    0.000427
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())